# **Análisis de Churn de Clientes**

Este proyecto aborda el desafío de identificar los factores que influyen en la cancelación de servicios por parte de los clientes. Utiliza datos reales extraídos desde una API en formato JSON, aplicando técnicas de ETL para su transformación y análisis. A través de visualizaciones estratégicas y un análisis exploratorio profundo, se generan insights clave para mejorar la retención de clientes.

## 1. Configuración del Entorno
Primero, importamos las librerías necesarias para el análisis y la visualización de datos.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json